# M1 FAULT 5

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\FAULT_M1_5


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto = 1

## RCOU

VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m1_fault5 = pd.read_csv("RCOU.csv")
rcou_m1_fault5 = rcou_m1_fault5.astype("float64")
rcou_m1_fault5 = rcou_m1_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1_fault5 = rcou_m1_fault5[((rcou_m1_fault5['C9'] >= potenza) & (rcou_m1_fault5['C10'] >= potenza) & (rcou_m1_fault5['C11'] >= potenza) & (rcou_m1_fault5['C12'] >= potenza) & (rcou_m1_fault5['C13'] >= potenza) & (rcou_m1_fault5['C14']>= potenza))]

rcou_m1_fault5 = rcou_m1_fault5.reset_index(drop=True)

display(rcou_m1_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,192480740.0,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0
1,192545491.0,1426.0,1445.0,1478.0,1390.0,1467.0,1403.0
2,192576646.0,1355.0,1538.0,1509.0,1394.0,1416.0,1490.0
3,192587140.0,1412.0,1504.0,1504.0,1412.0,1456.0,1465.0
4,192597516.0,1523.0,1403.0,1461.0,1472.0,1514.0,1414.0
...,...,...,...,...,...,...,...
7818,284579056.0,1623.0,1404.0,1545.0,1500.0,1468.0,1573.0
7819,284589530.0,1586.0,1441.0,1553.0,1480.0,1440.0,1586.0
7820,284599918.0,1557.0,1458.0,1561.0,1453.0,1425.0,1583.0
7821,284612550.0,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m1_fault5['TimeUS'][0]
max=rcou_m1_fault5['TimeUS'][len(rcou_m1_fault5 )-1]
print(max)
print(min)

284622968.0
192480740.0


## ATT & XFK

### XFK

XFK utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m1_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault5 = xkf1_m1_fault5.astype("float64")
xkf1_m1_fault5 = xkf1_m1_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault5 = xkf1_m1_fault5[((xkf1_m1_fault5['TimeUS'] >= min) & (xkf1_m1_fault5['TimeUS'] <= max))]
xkf1_m1_fault5 = xkf1_m1_fault5.reset_index(drop=True)
print(xkf1_m1_fault5)

           TimeUS  Roll  Pitch    Yaw
0     192488698.0  1.25   0.19  19.58
1     192499420.0  1.34   0.27  19.60
2     192510040.0  1.42   0.24  19.58
3     192521148.0  1.48   0.17  19.57
4     192532036.0  1.47   0.11  19.56
...           ...   ...    ...    ...
8332  284576961.0  1.35   0.18  19.19
8333  284586972.0  1.39   0.11  19.12
8334  284597792.0  1.44   0.02  19.05
8335  284610260.0  1.63  -0.17  18.93
8336  284620781.0  1.75  -0.32  18.85

[8337 rows x 4 columns]


### ATT

Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m1_fault5 = pd.read_csv("ATT.csv")
att_m1_fault5 = att_m1_fault5.astype("float64")
att_m1_fault5 = att_m1_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault5 = att_m1_fault5[((att_m1_fault5['TimeUS'] >= min) & (att_m1_fault5['TimeUS'] <= max))]
att_m1_fault5=att_m1_fault5.reset_index(drop=True)
print(att_m1_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624.0    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805.0    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203.0    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184.0    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737.0    -0.09  1.47      0.00   0.11   19.50  19.56
...           ...      ...   ...       ...    ...     ...    ...
8337  284578343.0     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791.0     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135.0     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707.0     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010.0     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XFK corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
if len(att_m1_fault5)>len(xkf1_m1_fault5):
    to_add=att_m1_fault5[len(xkf1_m1_fault5):]
    att_m1_fault5=att_m1_fault5[:len(xkf1_m1_fault5)]

if len(xkf1_m1_fault5)>len(att_m1_fault5):
    to_add=xkf1_m1_fault5[len(att_m1_fault5):]

for idx,i in enumerate(att_m1_fault5['Roll']):
    if(xkf1_m1_fault5['Roll'][idx] != i):
        att_m1_fault5['Roll'][idx] = ((att_m1_fault5['Roll'][idx] + xkf1_m1_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Pitch']):
    if(xkf1_m1_fault5['Pitch'][idx] != i):
        att_m1_fault5['Pitch'][idx] = ((att_m1_fault5['Pitch'][idx] + xkf1_m1_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Yaw']):
    if(xkf1_m1_fault5['Yaw'][idx] != i):
        att_m1_fault5['Yaw'][idx] = ((att_m1_fault5['Yaw'][idx] + xkf1_m1_fault5['Yaw'][idx])/2)

att_m1_fault5=pd.concat([att_m1_fault5,to_add])

print(att_m1_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624.0    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805.0    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203.0    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184.0    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737.0    -0.09  1.47      0.00   0.11   19.50  19.56
...           ...      ...   ...       ...    ...     ...    ...
8337  284578343.0     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791.0     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135.0     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707.0     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010.0     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


## IMU

VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault5 = imu_0_m1_fault5.astype("float64")
imu_0_m1_fault5 = imu_0_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault5 = imu_0_m1_fault5[((imu_0_m1_fault5['TimeUS'] >= min) & (imu_0_m1_fault5['TimeUS'] <= max))]

imu_1_m1_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault5 = imu_1_m1_fault5.astype("float64")
imu_1_m1_fault5 = imu_1_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault5 = imu_1_m1_fault5[((imu_1_m1_fault5['TimeUS'] >= min) & (imu_1_m1_fault5['TimeUS'] <= max))]

imu_2_m1_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault5 = imu_2_m1_fault5.astype("float64")
imu_2_m1_fault5 = imu_2_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault5 = imu_2_m1_fault5[((imu_2_m1_fault5['TimeUS'] >= min) & (imu_2_m1_fault5['TimeUS'] <= max))]

imu_m1_fault5 = pd.concat((imu_0_m1_fault5, imu_1_m1_fault5, imu_2_m1_fault5))
imu_m1_fault5=imu_m1_fault5.groupby(imu_m1_fault5.TimeUS, as_index=False).mean()

display(imu_m1_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,192482549.0,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212
1,192485022.0,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567
2,192488007.0,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817
3,192490421.0,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370
4,192492994.0,-0.168712,-0.092433,-0.014593,0.184634,-0.077262,-9.611569
...,...,...,...,...,...,...,...
31686,284611544.0,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003
31687,284614408.0,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220
31688,284617009.0,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360
31689,284619773.0,0.226640,-0.287556,-0.125083,-0.290287,-0.079025,-12.356240


## UNIONE DELLE TABELLE

Vengono unite le tabelle relative a IMU,ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m1_fault5 = pd.merge_ordered(imu_m1_fault5,att_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")
m1_fault5 = pd.merge_ordered(m1_fault5, rcou_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m1_fault5["TimeUS"] = m1_fault5["TimeUS"] - m1_fault5.iloc[0]["TimeUS"]
m1_fault5["TimeUS"] = m1_fault5["TimeUS"].astype(int)
m1_fault5["Guasto"] = guasto

In [ ]:
last_cell = m1_fault5.iloc[-1, m1_fault5.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [ ]:
m1_fault5_final = pd.merge_ordered(m1_fault5,df)
m1_fault5_final=m1_fault5_final.fillna(method="ffill").fillna(method="bfill")
m1_fault5_final = m1_fault5_final[m1_fault5_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [19]:
from datetime import timedelta
m1_fault5_final=m1_fault5_final.reset_index(drop=True)
m1_fault5_final['TimeUS'] = pd.to_datetime(m1_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,0.19,19.50,19.58,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
1,00:00:00.002857,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,0.19,19.50,19.58,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
2,00:00:00.005714,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567,-0.06,1.25,0.00,0.19,19.50,19.58,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
3,00:00:00.008571,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817,-0.06,1.25,0.00,0.19,19.50,19.58,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
4,00:00:00.011428,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370,-0.06,1.25,0.00,0.19,19.50,19.58,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32247,00:01:32.129679,0.118813,-0.206151,-0.138159,0.113017,-0.499642,-12.265113,4.67,1.44,-1.93,0.02,16.98,19.05,1557.0,1458.0,1561.0,1453.0,1425.0,1583.0,1.0
32248,00:01:32.132536,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003,4.66,1.63,-1.91,-0.17,16.98,18.93,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
32249,00:01:32.135393,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220,4.66,1.63,-1.91,-0.17,16.98,18.93,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
32250,00:01:32.138250,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360,4.66,1.63,-1.91,-0.17,16.98,18.93,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault5_final)/350)):
        V1=m1_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29872\2269437752.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29872\2269437752.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.018465,0.009846,2.594661,0.100790,60.0,0.0,4.590620,4.396057,-0.006336,0.006635,-0.525813,0.081587,101.0,72.0,2.652056,2.329331,0.087347,0.005059,-1.505082,0.112578,79.0,58.0,0.410686,0.262740,-0.054336,0.032751,-0.195341,0.188706,117.0,118.0,8.809698,6.491772,0.135931,0.068929,0.078632,0.295314,95.0,88.0,15.577873,11.404897,-10.578549,0.678407,-0.797301,10.610474,0.0,120.0,184.902914,26.712434,0.717806,0.201855,-1.407355,0.846479,95.0,92.0,9.619296,1.910001,0.380427,0.145570,-1.168019,0.538406,105.0,104.0,4.702035,2.581158,21.196239,3.082897,-1.164407,21.268631,111.0,110.0,2.099607,-0.149564,1570.962963,5526.555767,1.956859,1572.715944,105.0,104.0,1274.820003,814.328794,1507.193732,3234.830932,-0.776570,1508.263426,99.0,95.0,1005.840650,934.602497,1549.145299,2782.067399,0.032946,1550.040419,108.0,103.0,1025.294904,803.404238,1531.433048,4623.314790,0.687611,1532.937482,108.0,104.0,971.591546,540.069864,1526.079772,3344.176475,-0.292081,1527.171935,111.0,110.0,1872.055767,1162.546684,1553.598291,4921.041026,0.118677,1555.176733,112.0,109.0,778.385814,675.983069,1
1,1.0,-0.018733,0.009843,2.584391,0.100827,58.0,0.0,4.548868,4.256903,-0.006562,0.006634,-0.532179,0.081600,95.0,96.0,2.839400,1.847665,0.087370,0.005044,-1.500719,0.112533,73.0,51.0,0.474050,0.288089,-0.054986,0.032806,-0.208670,0.189041,119.0,78.0,8.518467,7.790583,0.136478,0.068838,0.083382,0.295413,98.0,91.0,15.710127,9.443775,-10.574614,0.681923,-0.790859,10.606718,0.0,111.0,186.085217,26.893932,0.716705,0.201708,-1.407525,0.845458,103.0,97.0,9.647742,2.040840,0.379375,0.145545,-1.167087,0.537641,97.0,96.0,4.499908,2.547822,21.205909,3.107028,-1.172426,21.278834,111.0,110.0,2.059969,-0.171055,1571.110795,5518.503359,1.971393,1572.861074,113.0,112.0,1305.600452,757.673857,1506.889205,3258.258345,-0.784726,1507.966869,110.0,106.0,1050.790675,992.321831,1549.184659,2774.686602,0.043350,1550.077389,113.0,111.0,1045.212053,669.183563,1531.275568,4618.872564,0.675229,1532.778724,115.0,111.0,1058.700799,516.934132,1526.073864,3334.661195,-0.284522,1527.162933,105.0,104.0,1972.303436,1076.051514,1553.497159,4910.621075,0.116215,1555.072376,103.0,100.0,869.609149,856.373957,1
2,2.0,-0.019001,0.009840,2.574363,0.100864,60.0,0.0,4.934244,4.117667,-0.006786,0.006633,-0.538388,0.081612,95.0,69.0,3.021038,2.033698,0.087393,0.005030,-1.496356,0.112488,71.0,59.0,0.538437,0.306656,-0.055633,0.032861,-0.221699,0.189374,115.0,82.0,8.198199,5.923846,0.137022,0.068747,0.088182,0.295513,83.0,76.0,15.797598,8.734105,-10.570703,0.685387,-0.784809,10.602981,0.0,110.0,187.249544,27.039597,0.715354,0.201778,-1.408962,0.844357,101.0,95.0,9.580174,2.035344,0.378244,0.145584,-1.166579,0.536880,101.0,100.0,4.285532,2.501331,21.215722,3.132195,-1.179196,21.2892

In [23]:
m1_fault5 = df_merged

## Output

Salviamo la tabella ottenuta in un file csv

In [24]:
path_file = path_file.replace(r"M1\FAULT_M1_5", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
